In [2]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [5]:
h=0.1
sphere1 = bempp.api.shapes.sphere(r = 0.4, h=h, origin=((0.5/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(r = 0.4, h=h, origin=(-((0.5/2) + 1), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

In [6]:
N_wn = 10
lbound_wn = 0   
ubound_wn = 10      
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

In [8]:
result_normal = []
for index2, point2 in enumerate(quad_points):

    wavenumber = -1j * np.log(point2)

    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat2 = np.block([[mat11,mat12_zero],[mat21_zero,mat22]])

    p,l,u = scipy.linalg.lu(mat)
    logdet_mat = 0
    for i in range(u.shape[0]):
        logdet_mat += cmath.log(u[i,i])

    p2,l2,u2 = scipy.linalg.lu(mat2)
    logdet_mat2 = 0
    for i in range(u2.shape[0]):
        logdet_mat2 += cmath.log(u2[i,i])

    logdet = logdet_mat - logdet_mat2
    result_normal.append(logdet)
    print (logdet)

0j
(-8.543932381144259e-06+0j)
(-9.983519521483686e-05+0j)
(-0.00042984534593415447+0j)
(-0.0012295736496525933+0j)
(-0.002812439008266665+0j)
(-0.0055879731798995635+0j)
(-0.010081682189593266+0j)
(-0.016962924360086618+0j)
(-0.02708430947677698+0j)


In [11]:
casimir_energy = np.trapz(-np.array(result_normal) / quad_points, quad_points) / (2 * np.pi)
print(casimir_energy)

(0.001124876650892324+0j)


In [12]:
results = []

for wavenumber in range(10):

    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat2 = np.block([[mat11,mat12_zero],[mat21_zero,mat22]])

    p,l,u = scipy.linalg.lu(mat)
    logdet_mat = 0
    for i in range(u.shape[0]):
        logdet_mat += cmath.log(u[i,i])

    p2,l2,u2 = scipy.linalg.lu(mat2)
    logdet_mat2 = 0
    for i in range(u2.shape[0]):
        logdet_mat2 += cmath.log(u2[i,i])

    logdet = logdet_mat - logdet_mat2
    results.append(logdet)
    print (logdet)

(-0.02708430947677698+0j)
(0.015922657358714787+0.013588268535108528j)
(-0.007618983283464331-0.012763515663970715j)
(0.004541810591035755+0.012070076757241921j)
(-0.0012982813923372305-0.011805638151917819j)
(-0.0011062915737056755+0.011039307342297633j)
(0.0035845182292177924-0.010086123938489067j)
(-0.005657187441101996+0.00873873251259738j)
(0.0073981009036288015-0.006891155203387456j)
(-0.00876786442950106+0.004886613986961663j)


In [13]:
np.trapz(np.imag(results), range(10))/(2*np.pi)

0.0010079695048504193

In [14]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*wavenumber*r))*scipy.special.jv(q+0.5, wavenumber*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [16]:
combined_scatters = bempp.api.grid.grid.union([sphere1,sphere2])
space_combined = bempp.api.function_space(combined_scatters,'P',1)

unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_unit_sphere = unit_sphere.vertices
space_unit_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

In [17]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere, space_unit_sphere, space_unit_sphere).weak_form().A

In [19]:
sph_harm_coeff = []

deg = 5

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere, fun = sph_harm)
        sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))

In [20]:
for index, point in enumerate([0.05, 1, 2, 3, 4, 5, 6, 7, 8, 9]):
    wavenumber = point
    eta = wavenumber

    identity1 = bempp.api.operators.boundary.sparse.identity(space_sphere1, space_sphere1, space_sphere1)
    slp1 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    dlp1 = bempp.api.operators.boundary.helmholtz.double_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp_far1 = helmholtz_farfield.single_layer(space_sphere1, vert_unit_sphere, wavenumber)
    dlp_far1 = helmholtz_farfield.double_layer(space_sphere1, vert_unit_sphere, wavenumber)

    lhs1 = 0.5 * identity1 + dlp1 - 1j* eta *slp1

    identity2 = bempp.api.operators.boundary.sparse.identity(space_sphere2, space_sphere2, space_sphere2)
    slp2 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2, space_sphere2, space_sphere2, wavenumber)
    dlp2 = bempp.api.operators.boundary.helmholtz.double_layer(space_sphere2, space_sphere2, space_sphere2, wavenumber)
    slp_far2 = helmholtz_farfield.single_layer(space_sphere2, vert_unit_sphere, wavenumber)
    dlp_far2 = helmholtz_farfield.double_layer(space_sphere2, vert_unit_sphere, wavenumber)

    lhs2 = 0.5 * identity2 + dlp2 - 1j* eta *slp2

    identity = bempp.api.operators.boundary.sparse.identity(space_combined, space_combined, space_combined)
    slp = bempp.api.operators.boundary.helmholtz.single_layer(space_combined, space_combined, space_combined, wavenumber)
    dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_combined, space_combined, space_combined, wavenumber)
    slp_far = helmholtz_farfield.single_layer(space_combined, vert_unit_sphere, wavenumber)
    dlp_far = helmholtz_farfield.double_layer(space_combined, vert_unit_sphere, wavenumber)

    lhs = 0.5 * identity + dlp - 1j* eta *slp
    
    #===================================================================================================================
    slp_far_field_coeff1 = []
    slp_far_field_coeff2 = []
    slp_far_field_coeff = []

    for q in range(deg+1):
        for p in range(-q,q+1):
            @bempp.api.complex_callable
            def dirichlet_fun(x,n,domain_index,result):
                    with objmode():
                        result[0] = - regular_spherical_wavefunctions(p,q,x)

            rhs_fun1 = bempp.api.GridFunction(space_sphere1, fun = dirichlet_fun)
            field1, info1 = bempp.api.linalg.gmres(lhs1, rhs_fun1, tol=1E-8)
            slp_far_field_coeff1.append(dlp_far1.evaluate(field1) - 1j * eta * slp_far1.evaluate(field1))

            rhs_fun2 = bempp.api.GridFunction(space_sphere2, fun = dirichlet_fun)
            field2, info2 = bempp.api.linalg.gmres(lhs2, rhs_fun2, tol=1E-8)
            slp_far_field_coeff2.append(dlp_far2.evaluate(field2) - 1j * eta * slp_far2.evaluate(field2))

            rhs_fun = bempp.api.GridFunction(space_combined, fun = dirichlet_fun)
            field, info = bempp.api.linalg.gmres(lhs, rhs_fun, tol=1E-8)
            slp_far_field_coeff.append(dlp_far.evaluate(field) - 1j * eta * slp_far.evaluate(field))
            
    #====================================================================================================
    T_matrix1 = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
    for i in range((deg+1)**2):
        for j in range((deg+1)**2):
            T_matrix1[i,j] = (slp_far_field_coeff1[j] @ mass_mat @ sph_harm_coeff[i])*wavenumber

    T_matrix2 = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
    for i in range((deg+1)**2):
        for j in range((deg+1)**2):
            T_matrix2[i,j] = (slp_far_field_coeff2[j] @ mass_mat @ sph_harm_coeff[i])*wavenumber

    T_matrix = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
    for i in range((deg+1)**2):
        for j in range((deg+1)**2):
            T_matrix[i,j] = (slp_far_field_coeff[j] @ mass_mat @ sph_harm_coeff[i])*wavenumber

    S_matrix = np.identity(T_matrix.shape[0]) + 2*T_matrix
    S_matrix1 = np.identity(T_matrix1.shape[0]) + 2*T_matrix1
    S_matrix2 = np.identity(T_matrix2.shape[0]) + 2*T_matrix2

    rhs_value_S =  (np.log(np.linalg.det(S_matrix))-(np.log(np.linalg.det(S_matrix1)) + np.log(np.linalg.det(S_matrix2))))/(2*np.pi*1j)
    print(wavenumber, rhs_value_S)

0.05 (0.001713848791710081-7.785147737242081e-08j)
1 (-0.004343193124303476+2.02509192185574e-05j)
2 (1.0040364535561148-2.350658320484085e-05j)
3 (-1.0038388203634176-4.204779615932351e-05j)
4 (0.0072416515615332895-0.003182977000323236j)
5 (-0.9650317318724645+0.0014203639380352855j)
6 (-0.004405598182765841-0.08892845646285269j)
7 (-0.028600472176576082-0.06634594929133053j)
8 (-0.5209906096080763-0.7403720447590881j)
9 (-1.1132139277798332-0.03997891457798152j)


In [22]:
result_rhs = [0.001713848791710081, -0.004343193124303476, 1.0040364535561148, -1.0038388203634176, 0.0072416515615332895,
              -0.9650317318724645, -0.004405598182765841, -0.028600472176576082, -0.5209906096080763, -1.1132139277798332]

In [23]:
np.trapz(result_rhs, [0.05, 1, 2, 3, 4, 5, 6, 7, 8, 9])/2

-1.0358083130478513